<a href="https://colab.research.google.com/github/polivio/PEE/blob/master/DecisionTree_KDD_BR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# https://www.kaggle.com/c/kddbr-2019/data

# KNN KDD BR -Polivio
import numpy as np
from sklearn import preprocessing, neighbors
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.model_selection import cross_val_score



# preprocessing # blob # raw
train = pd.read_csv('https://github.com/polivio/PEE/raw/master/Training_grouped.csv')
test = pd.read_csv('https://github.com/polivio/PEE/raw/master/Test_grouped.csv')


train.info()

#train.isna().sum()
##X                            4592
##XY                          19463
##Y                            3711

# Dica Mateus substituição pela média
train['X'].fillna(train['X'].mean(),inplace=True)
train['XY'].fillna(train['XY'].mean(),inplace=True)
train['Y'].fillna(train['XY'].mean(),inplace=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80000 entries, 0 to 79999
Data columns (total 21 columns):
Unnamed: 0                          80000 non-null int64
traininglabelorder.scatterplotID    80000 non-null int64
qtde.ponto                          80000 non-null int64
Negative                            80000 non-null int64
Sample                              80000 non-null int64
mean.signalX                        80000 non-null float64
sd.signalX                          80000 non-null float64
var.signalX                         80000 non-null float64
kurtosis.signalX                    80000 non-null float64
skew.signalX                        80000 non-null float64
mean.signalY                        80000 non-null float64
sd.signalY                          80000 non-null float64
var.signalY                         80000 non-null float64
kurtosis.signalY                    80000 non-null float64
skew.signalY                        80000 non-null float64
L               

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80000 entries, 0 to 79999
Data columns (total 21 columns):
Unnamed: 0                          80000 non-null int64
traininglabelorder.scatterplotID    80000 non-null int64
qtde.ponto                          80000 non-null int64
Negative                            80000 non-null int64
Sample                              80000 non-null int64
mean.signalX                        80000 non-null float64
sd.signalX                          80000 non-null float64
var.signalX                         80000 non-null float64
kurtosis.signalX                    80000 non-null float64
skew.signalX                        80000 non-null float64
mean.signalY                        80000 non-null float64
sd.signalY                          80000 non-null float64
var.signalY                         80000 non-null float64
kurtosis.signalY                    80000 non-null float64
skew.signalY                        80000 non-null float64
L               

In [5]:
train = train.loc[:,[#'qtde.ponto', 
       'mean.signalX', 'sd.signalX', 'kurtosis.signalX',
      'mean.signalY', 'sd.signalY', 
       'kurtosis.signalY', 'L', 'X', 'XY', 'Y',
       'mean.silhouette', 'traininglabelorder.score']]

## Quando iremos usar os dados de teste?
test = test.loc[:,[# 'qtde.ponto', 
       'mean.signalX', 'sd.signalX', 'kurtosis.signalX',
      'mean.signalY', 'sd.signalY', 
       'kurtosis.signalY', 'L', 'X', 'XY', 'Y',
       'mean.silhouette']]

train.head(20)







,mean.signalX,sd.signalX,kurtosis.signalX,mean.signalY,sd.signalY,kurtosis.signalY,L,X,XY,Y,mean.silhouette,traininglabelorder.score
0,3164.835616,804.757625,3.516731,1586.430137,568.651580,3.170632,4,92.0,269.000000,80.421065,0.840146,0.009647
1,1737.776042,944.798845,2.475172,2999.940104,1716.790542,2.195067,4,82.0,49.000000,249.000000,0.964427,0.842084
2,1139.621984,866.460225,1.922726,1836.640751,950.478945,1.436889,4,156.0,7.000000,206.000000,0.936210,0.744601
3,3609.692708,1808.877599,1.985210,2469.164062,1219.231193,1.913381,4,100.0,190.000000,90.000000,0.896070,0.842084
4,1867.203655,938.232021,2.176598,2180.908616,1515.205489,1.155226,4,176.0,2.000000,201.000000,0.952810,0.944177
5,449.072917,507.413489,29.834375,2910.351562,358.648637,7.460460,4,12.0,80.421065,368.000000,0.968487,1.000000
6,2076.953039,655.688216,2.774567,1170.375691,342.483082,2.222238,7,132.0,117.000000,106.000000,0.913787,0.578706
7,830.200000,648.128942,9.725987,3241.095000,693.599022,5.545654,5,3.0,22.000000,170.000000,0.916503,1.000000
8,3586.317708,1862.356219,2.630635,1683.195312,1436.342429,6.643115,4,326.0,80.421065,54.000000,0.956756,0.944319
9,222.168831,270.887643,23.916200,2146.623377,434.285275,4.516220,2,6.0,2.000000,144.000000,0.949731,0.751477


In [0]:
train.head(20)
# change the dtype to 'float64' 
train = trainP.astype('float64')
trainP.info()



In [0]:
# feature qtde.ponto poderá ser removida??


X = np.array(trainP.drop(['traininglabelorder.score'], 1))

y = np.array(trainP['traininglabelorder.score'])


print(y)


# X - features, y - lables
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


# Regressor
clf = DecisionTreeRegressor(max_depth = None)

clf.fit(X_train , y_train)

accuracy = clf.score(X_test, y_test)



In [0]:
print(cross_val_score(clf, X, y, cv=5))
print(cross_val_score(clf, X, y, cv=5).mean())

tree.plot_tree(clf.fit(X_train , y_train))